# Downloading packages

In [ ]:
!python -m pip install pyyaml
import sys, os, distutils.core

In [ ]:
!pip install torch

In [ ]:
# Detectron 2
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!pip install torchvision

In [ ]:
!pip install opencv-python

# Import Downloaded packages

In [6]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

/usr/bin/sh: 1: nvcc: not found
torch:  2.6 ; cuda:  cu124
detectron2: 0.6


In [7]:
# Andre pakker

# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, random
import pandas as pd

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [8]:
import os
import cv2
from detectron2.data import DatasetCatalog, MetadataCatalog
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import convert_to_coco_json
import shutil
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Import images from GitHub

In [9]:
from getpass import getpass
import subprocess

# Get GitHub token
token = getpass("Enter your GitHub token: ")

# Define repository URL
repo_url = f"https://{token}@github.com/casperbak1/Dataprojekt.git"

# Kloner main branch, og kun seneste commit (depth 1)
subprocess.run(["git", "clone", "--branch", "main", "--depth", "1", repo_url])

repo_path = "Dataprojekt"
if os.path.exists(repo_path):  # Hvis der findes en GitHub sti "Dataprojekt"
    subprocess.run(["git", "sparse-checkout", "init", "--cone"], cwd=repo_path) # Så klon mappen "Data/Clean Data/Overbite Data"
    subprocess.run(["git", "sparse-checkout", "set", "Data/Clean Data/Overbite Data"], cwd=repo_path)

print("Repository cloned with only the 'Overbite Data' folder.")

Enter your GitHub token:  ········


Repository cloned with only the 'Overbite Data' folder.


# Initialise the training data

In [10]:
# Load the annotations from the CSV file
ANNOTATIONS_FILE = "Dataprojekt/Data/Clean Data/Overbite Data/Updated_Labels.csv"
annotations_df = pd.read_csv(ANNOTATIONS_FILE)

# Sti til dataen
DATASET_PATH = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Data"

def my_dataset_function():
    dataset_dicts = []

    # Group annotations by filename (in case multiple keypoints exist for an image)
    grouped_annotations = annotations_df.groupby("Filename")

    for idx, filename in enumerate(os.listdir(DATASET_PATH)):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            record = {}
            file_path = os.path.join(DATASET_PATH, filename)

            # Read image dimensions
            height, width = cv2.imread(file_path).shape[:2]

            # Initialize the dataset record
            record["file_name"] = file_path
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width

            # Default empty annotation list
            record["annotations"] = []

            # Check if the file has keypoint annotations
            if filename in grouped_annotations.groups:
                keypoints_list = []
                for _, row in grouped_annotations.get_group(filename).iterrows():
                    x, y = row["X"], row["Y"]
                    keypoints_list.append(x)  # X-coordinate
                    keypoints_list.append(y)  # Y-coordinate
                    keypoints_list.append(2)  # Visibility flag (0=not visible, 1=occluded, 2=visible)

                # Create the annotation entry
                annotation = {
                    "bbox": [0, 0, width, height],  # Dummy bbox covering entire image
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": 0,  # If you have multiple classes (Vi har 1)
                    "keypoints": keypoints_list,
                    "num_keypoints": len(keypoints_list) // 3
                }
                record["annotations"].append(annotation)

            dataset_dicts.append(record)

    return dataset_dicts

# Register the dataset
DatasetCatalog.register("Overbite_Data", my_dataset_function)
MetadataCatalog.get("Overbite_Data").set(
    thing_classes=["object"],  # Modify for actual class names (Mangler godt navn)
    keypoint_names=["keypoint"],  # Name of keypoints (Mangler endnu bedre navn)
    keypoint_flip_map=[]  # Add keypoint flip pairs if needed (Nope)
)

# Test if it works
dataset_dicts = DatasetCatalog.get("Overbite_Data")
print(f"Loaded {len(dataset_dicts)} images with keypoints.")

Loaded 1600 images with keypoints.


# Initialise the test data

In [11]:
# Funktion til at hente verifikationsdata
def my_validation_function():
    dataset_dicts = []
    DATASET_PATH = "Dataprojekt/Data/Clean Data/Overbite Data/Annotated Verification Data"

    grouped_annotations = annotations_df.groupby("Filename")

    for idx, filename in enumerate(os.listdir(DATASET_PATH)):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            record = {}
            file_path = os.path.join(DATASET_PATH, filename)
            height, width = cv2.imread(file_path).shape[:2]

            record["file_name"] = file_path
            record["image_id"] = idx
            record["height"] = height
            record["width"] = width
            record["annotations"] = []

            if filename in grouped_annotations.groups:
                keypoints_list = []
                for _, row in grouped_annotations.get_group(filename).iterrows():
                    x, y = row["X"], row["Y"]
                    keypoints_list.append(x)
                    keypoints_list.append(y)
                    keypoints_list.append(2)  # Visibility flag

                annotation = {
                    "bbox": [0, 0, width, height],  # Dummy bbox
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "category_id": 0,
                    "keypoints": keypoints_list,
                    "num_keypoints": len(keypoints_list) // 3
                }
                record["annotations"].append(annotation)

            dataset_dicts.append(record)

    return dataset_dicts

# Registrér valideringsdatasæt
DatasetCatalog.register("Overbite_Validation", my_validation_function)
MetadataCatalog.get("Overbite_Validation").set(
    thing_classes=["object"],
    keypoint_names=["keypoint"],
    keypoint_flip_map=[]
)

namespace(name='Overbite_Validation',
          thing_classes=['object'],
          keypoint_names=['keypoint'],
          keypoint_flip_map=[])

# Train the model

In [12]:
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
from detectron2.data.datasets import convert_to_coco_json

# Create the configuration
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml")

cfg.DATASETS.TRAIN = ("Overbite_Data",)  # Training dataset
cfg.DATASETS.TEST = ("Overbite_Validation",)  # Validation dataset

cfg.DATALOADER.NUM_WORKERS = 8
cfg.SOLVER.IMS_PER_BATCH = 12  # Batch size (Antal billeder per batch) (Større skulle gerne give god træning, men vi har begrænset GPU i "Colab")
cfg.SOLVER.BASE_LR = 0.0005  # Learning rate
cfg.SOLVER.MAX_ITER = 20000  # Antal træningsiterationer (5000 = Ca. 30 min)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128 # Antal "Regions Of Interest" per billede (Høj = Langsom træning, mere stabil. Lav = Hurtig træning, mere ustabil)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # Antal klasser
cfg.MODEL.KEYPOINT_ON = True  # Enable keypoint detection
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 1  # Antal keypoints


cfg.TEST.KEYPOINT_OKS_SIGMAS = [0.1] # Set OKS sigma for 1 keypoint (Default is 17 for COCO)

# Automatisk validering hver 500. iteration
cfg.TEST.EVAL_PERIOD = 500

# Output directory for the model
cfg.OUTPUT_DIR = "./output/Overbite_Model"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Konverter valideringsettet til COCO JSON format
coco_annotation_path = os.path.join(cfg.OUTPUT_DIR, "Overbite_Validation_coco_format.json")
convert_to_coco_json("Overbite_Validation", coco_annotation_path)
print(f"Validation set converted to COCO format: {coco_annotation_path}")

# Inkluder evaluatoren
class TrainerWithEval(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR)

# Træn modellen med evaluering
trainer = TrainerWithEval(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Kør en sidste evaluering på valideringsdataen
val_loader = build_detection_test_loader(cfg, "Overbite_Validation")
inference_on_dataset(trainer.model, val_loader, COCOEvaluator("Overbite_Validation", cfg, False, output_dir=cfg.OUTPUT_DIR))

print("Training and validation completed!")

WARNING [03/24 14:27:00 d2.data.datasets.coco]: Using previously cached COCO format annotations at './output/Overbite_Model/Overbite_Validation_coco_format.json'. You need to clear the cache file if your dataset has been modified.
Validation set converted to COCO format: ./output/Overbite_Model/Overbite_Validation_coco_format.json
[03/24 14:27:02 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, 

Skip loading parameter 'roi_heads.keypoint_head.score_lowres.weight' to the model due to incompatible shapes: (512, 17, 4, 4) in the checkpoint but (512, 1, 4, 4) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.keypoint_head.score_lowres.bias' to the model due to incompatible shapes: (17,) in the checkpoint but (1,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.keypoint_head.score_lowres.{bias, weight}


[03/24 14:27:16 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.12/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/opt/conda/lib/python3.12/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  keypoints = torch.as_tensor(keypoints, dtype=torch.float32, device=device)
/opt/conda/lib/python3.12/site-packages/detectron2/structures/keypoints.py:29: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. P

[03/24 14:27:23 d2.utils.events]:  eta: 1:47:33  iter: 19  total_loss: 9.102  loss_cls: 0.3479  loss_box_reg: 0.3048  loss_keypoint: 8.19  loss_rpn_cls: 0.2242  loss_rpn_loc: 0.04569    time: 0.3210  last_time: 0.3245  data_time: 0.0224  last_data_time: 0.0159   lr: 9.9905e-06  max_mem: 17882M
[03/24 14:27:29 d2.utils.events]:  eta: 1:47:17  iter: 39  total_loss: 8.545  loss_cls: 0.1495  loss_box_reg: 0.24  loss_keypoint: 8.015  loss_rpn_cls: 0.08935  loss_rpn_loc: 0.03311    time: 0.3217  last_time: 0.3226  data_time: 0.0159  last_data_time: 0.0160   lr: 1.998e-05  max_mem: 17882M
[03/24 14:27:36 d2.utils.events]:  eta: 1:47:03  iter: 59  total_loss: 7.811  loss_cls: 0.0293  loss_box_reg: 0.08388  loss_keypoint: 7.664  loss_rpn_cls: 0.0134  loss_rpn_loc: 0.0235    time: 0.3216  last_time: 0.3225  data_time: 0.0159  last_data_time: 0.0160   lr: 2.997e-05  max_mem: 17883M
[03/24 14:27:42 d2.utils.events]:  eta: 1:47:04  iter: 79  total_loss: 6.595  loss_cls: 0.01835  loss_box_reg: 0.041